In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data.dataloader
import torchvision.io
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.io
import pandas as pd
from skimage import io

In [ ]:
num_workers = 0
batch_size = 32

In [ ]:
def get_mean_and_std(loader):
    mean = 0
    std = 0
    total_images_count = 0
    for images, _ in loader:
        image_count_in_a_batch = images.size(0)
        images = images.view(image_count_in_a_batch, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += image_count_in_a_batch
    mean /= total_images_count
    std /= total_images_count
    return mean, std

In [ ]:
def jaccard(arg1, arg2):
    arg1 = arg1.detach().numpy()
    arg2 = arg2.detach().numpy()
    intersection = np.logical_and(arg1, arg2)
    union = np.logical_or(arg1, arg2)
    similarity = intersection.sum()/float(union.sum())
    return similarity

In [ ]:
def imshow2(inp, title=None):
    mean_nums = [0.5, 0.5, 0.5, 0.5]
    std_nums = [0.5, 0.5, 0.5, 0.5]
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([mean_nums])
    std = np.array([std_nums])
    inp = (std*inp)+mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [ ]:
class ISIC2017DatasetClass(torch.utils.data.Dataset):
    def __init__(self, csv_file, train_dir, mask_dir, transform=None, mask_transform=None, train_transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.train_dir = train_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        self.train_transform = train_transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        mask_img_path = os.path.join(self.mask_dir, self.annotations.iloc[index, 0])
        train_image_path = os.path.join(self.train_dir, self.annotations.iloc[index, 0].replace("Pred", "Training_Data"))

        mask_img_path = mask_img_path + ".png"
        mask_image = io.imread(mask_img_path)
        if self.mask_transform:
            mask_image = self.mask_transform(mask_image)

        train_image_path = train_image_path + ".jpg"
        train_image = io.imread(train_image_path)
        if self.train_transform:
            train_image = self.train_transform(train_image)

        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        image = torch.cat((train_image, mask_image), 0)

        if self.transform:
            image = self.transform(image)

        return image, y_label

In [ ]:
train_dataset_path = 'drive/MyDrive/Pesquisa/DataBase/Train'
train_pred_dataset_path = 'drive/MyDrive/Pesquisa/DataBase/Train_Pred'

test_dataset_path = 'drive/MyDrive/Pesquisa/DataBase/Validation'
test_pred_dataset_path = 'drive/MyDrive/Pesquisa/DataBase/Validation_Pred'

csv_pred_path = 'drive/MyDrive/Pesquisa/DataBase/labels_train_threshold_07.csv'
csv_validation_pred_path = 'drive/MyDrive/Pesquisa/DataBase/labels_validation_threshold_0_7.csv'

In [ ]:
train_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

mask_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])

valid_train_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)),
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

valid_mask_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])

transform = None #transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomRotation(10)])

In [ ]:
VGG_arc = [32, 32, 'M', 64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 1024, 1024]


class VGG16(nn.Module):

    def __init__(self, in_channels=3):
        super(VGG16, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_arc)
        self.l1 = nn.Linear(1024, 1)
        self.avg = nn.AvgPool2d((7, 7))
        self.softmax = nn.Softmax()

    def forward(self, x):# (32x4x224x224)
        x = self.avg(self.conv_layers(x))# (32x1024x1x1)
        x = x.view(x.shape[0], -1).detach()# (32x1024)
        x = F.relu(self.l1(x))# (32x1)
        x = self.softmax(x.view(-1))# (32)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for i in architecture:
            if type(i) == int:
                out_channels = i

                layers += [nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), padding=1), nn.BatchNorm2d(i), nn.ReLU()]
                in_channels = i
            elif i == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        return nn.Sequential(*layers)

In [ ]:
train_dataset = ISIC2017DatasetClass(train_dir=train_dataset_path, mask_dir=train_pred_dataset_path,
                                                transform=transform, csv_file=csv_pred_path,
                                                train_transform=train_transform, mask_transform=mask_transform)

dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = ISIC2017DatasetClass(train_dir=test_dataset_path, mask_dir=test_pred_dataset_path,
                                                     transform=transform, csv_file=csv_validation_pred_path,
                                                     train_transform=valid_train_transform, mask_transform=valid_mask_transform)

valid_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:

train_on_gpu = torch.cuda.is_available()

model = VGG16(in_channels=4) #CRIANDO O MODELO

#TESTES DE PARAMETROS E FUNCIONAMENTO DA ARQUITETURA DA REDE

if train_on_gpu:
  print("Cuda ATIVO!")
  model.cuda()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
epochs = 30
valid_loss_min = np.Inf

pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

18879425


In [ ]:
if __name__ == '__main__':

    print("Inicio do Treinamento")
    for epoch in range(epochs):

        train_loss = 0.0
        valid_loss = 0.0

        model.train()

        for data, target in dataset_loader:
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            output = model.forward(data).detach().double()
            target = target.detach().double()
            loss = criterion(output, target)
            loss.requires_grad = True
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)

        model.eval()

        for valid_data, valid_target in valid_loader:

            if train_on_gpu:
                valid_data, valid_target = valid_data.cuda(), valid_target.cuda()

            output = model.forward(valid_data).detach().double()
            valid_target = valid_target.detach().double()

            loss = criterion(output, valid_target)
            valid_loss += loss.item()*valid_data.size(0)

        train_loss = train_loss/len(dataset_loader.sampler)
        valid_loss = valid_loss/len(valid_loader.sampler)

        print('Epoch: {} \tTraining Loss: {:.6f}  \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} ---> {:.6f}). Saving Model ...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), 'model_cifar.pt')
            valid_loss_min = valid_loss

Inicio do Treinamento


FileNotFoundError: ignored